In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import os
import pandas as pd
import scanpy as sc
import cellink as cl

/data/nasif12/home_if12/l_mai/.conda/envs/mm_scgenetics/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from cellink.tl._burden_testing import *

In [4]:
cell_type_col = "cell_label"

In [5]:
base_data_dir = Path("/s/project/sys_gen_students/2024_2025/project04_rare_variant_sc/")
input_dir = base_data_dir / "input_data"
output_dir = base_data_dir / "output"
## defining the path to the data
scdata_path = input_dir / "chr22_OneK1K_cohort_gene_expression_matrix_14_celltypes.h5ad.gz"
gdata_path = input_dir / "plink/"
dump_path = base_data_dir / "output/example_output"
vep_anno_file = input_dir /  "all_variants_annotated_fixed_ref_reversed_id.txt"
DNA_LM_file = input_dir/ "onek1k_inf_scores_upstream_model.tsv"
## defining target chromosome
target_chromosome = "22"

In [6]:
# data
data = pd.read_pickle(output_dir/"data.pkl")

In [9]:
data.gdata.varm["annotations_0"]

gnomADe_EAS_AF Feature_type  Consequence_intergenic_variant  \
snp_id                                                                         
22_16849573_G_A             NaN            -                               1   
22_16849971_T_A             NaN            -                               1   
22_16850437_A_G             NaN            -                               1   
22_16851225_T_C             NaN            -                               1   
22_16851356_T_C             NaN            -                               1   
...                         ...          ...                             ...   
22_51202748_G_A             NaN   Transcript                               0   
22_51208568_T_G             NaN   Transcript                               0   
22_51211031_G_A             NaN   Transcript                               0   
22_51213613_T_C             NaN   Transcript                               0   
22_51216564_C_T             NaN   Transcript                               0   

                PHENO  Consequence_start_lost  gnomADe_AMR_AF  \
snp_id                                                          
22_16849573_G_A     -                       0             NaN   
22_16849971_T_A     -                       0             NaN   
22_16850437_A_G     -                       0             NaN   
22_16851225_T_C     -                       0             NaN   
22_16851356_T_C     -                       0             NaN   
...               ...                     ...             ...   
22_51202748_G_A     -                       0             NaN   
22_51208568_T_G     -                       0             NaN   
22_51211031_G_A     -                       0             NaN   
22_51213613_T_C     -                       0             NaN   
22_51216564_C_T     -                       0             NaN   

                Existing_variation CLIN_SIG  CADD_RAW  gnomADe_NFE_AF  ...  \
snp_id                                                                 ...   
22_16849573_G_A        rs112435201        -  0.433139             NaN  ...   
22_16849971_T_A          rs7287956        -  0.442607             NaN  ...   
22_16850437_A_G          rs5748209        -  0.369731             NaN  ...   
22_16851225_T_C          rs5746874        -  0.393139             NaN  ...   
22_16851356_T_C          rs5748581        -  0.377289             NaN  ...   
...                            ...      ...       ...             ...  ...   
22_51202748_G_A          rs9616963        -  0.550962             NaN  ...   
22_51208568_T_G        rs148425445        -  0.102244             NaN  ...   
22_51211031_G_A          rs9616968        -  0.153297             NaN  ...   
22_51213613_T_C         rs34726907        - -0.394121             NaN  ...   
22_51216564_C_T          rs9616970        - -0.113869             NaN  ...   

                 Consequence_splice_donor_5th_base_variant  \
snp_id                                                       
22_16849573_G_A                                          0   
22_16849971_T_A                                          0   
22_16850437_A_G                                          0   
22_16851225_T_C                                          0   
22_16851356_T_C                                          0   
...                                                    ...   
22_51202748_G_A                                          0   
22_51208568_T_G                                          0   
22_51211031_G_A                                          0   
22_51213613_T_C                                          0   
22_51216564_C_T                                          0   

                 Consequence_splice_donor_region_variant  \
snp_id                                                     
22_16849573_G_A                                        0   
22_16849971_T_A                                        0   
22_16850437_A_G                                        0   
22_16851

In [ ]:
data.gdata.varm["annotations_0"]

### Understanding burden computation

In [32]:
this_vars = data.gdata.varm["annotations_0"][data.gdata.varm["annotations_0"]["Gene"] == "ENSG00000184319"].index
this_vars


Index(['22_51197576_C_G', '22_51198868_C_T', '22_51198906_A_G',
       '22_51198998_A_C', '22_51202748_G_A', '22_51208568_T_G',
       '22_51211031_G_A', '22_51213613_T_C', '22_51216564_C_T'],
      dtype='object', name='snp_id')

In [24]:
data.gdata.varm["annotations_0"]["Gene"]

snp_id
22_16849573_G_A                  -
22_16849971_T_A                  -
22_16850437_A_G                  -
22_16851225_T_C                  -
22_16851356_T_C                  -
                        ...       
22_51202748_G_A    ENSG00000184319
22_51208568_T_G    ENSG00000184319
22_51211031_G_A    ENSG00000184319
22_51213613_T_C    ENSG00000184319
22_51216564_C_T    ENSG00000184319
Name: Gene, Length: 96257, dtype: object

In [28]:
data.gdata.varm["annotations_0"]["Location"]

snp_id
22_16849573_G_A    22:16849573
22_16849971_T_A    22:16849971
22_16850437_A_G    22:16850437
22_16851225_T_C    22:16851225
22_16851356_T_C    22:16851356
                      ...     
22_51202748_G_A    22:51202748
22_51208568_T_G    22:51208568
22_51211031_G_A    22:51211031
22_51213613_T_C    22:51213613
22_51216564_C_T    22:51216564
Name: Location, Length: 96257, dtype: object

In [46]:
## we need the location of the gene-> choose vars based on window
#todo: test +/- strand

In [17]:
import requests

def get_gene_location(ensembl_id):
    url = f"https://rest.ensembl.org/lookup/id/{ensembl_id}?content-type=application/json"
    response = requests.get(url)
    if response.ok:
        data = response.json()
        #print(data)
        if data.get('strand') == 1: # forward
            return f"{data.get('seq_region_name')}:{data.get('start')}-{data.get('end')}"
        else: # reverse
            return f"{data.get('seq_region_name')}:{data.get('end')}-{data.get('start')}"
    else:
        return f"Error: {response.status_code}, {response.text}"


In [20]:
#eg: forward
gene_id = "ENSG00000184319"
location = get_gene_location(gene_id)
print(location)

#eg: rev
gene_id = "ENSG00000118137"
location = get_gene_location(gene_id)
print(location)


22:50798655-50799123
11:116837622-116835751


In [44]:
import pandas as pd

def find_snps_near_gene(gdata, gene_location, bp_range=10000):
    """
    Finds SNPs within a specified range of a gene's location.

    Parameters:
        gdata (pd.DataFrame): DataFrame with a 'Location' column in the format "chromosome:position".
        gene_location (str): Gene location in the format "chromosome:start-end".
        bp_range (int): Range in base pairs to search upstream and downstream.

    Returns:
        pd.Index: Index of SNPs within the specified range.
    """
    # Parse the gene location
    gene_chrom, gene_range = gene_location.split(":")
    gene_start, gene_end = map(int, gene_range.split("-"))

    # Extract chromosome and position from the SNPs
    gdata[['Chromosome', 'Position']] = gdata['Location'].str.split(':', expand=True)
    gdata['Position'] = gdata['Position'].astype(int)

    # Filter for SNPs within the range
    snps_in_range = gdata[
        (gdata['Chromosome'] == gene_chrom) &
        (gdata['Position'] >= gene_start - bp_range) &
        (gdata['Position'] <= gene_end + bp_range)
    ]

    return snps_in_range.index


In [45]:

# Find SNPs within 10,000 bp of the gene
result = find_snps_near_gene(data.gdata.varm["annotations_0"], location, bp_range=10000)
print(result)


Index(['22_50788696_A_G', '22_50788848_T_C', '22_50789302_T_C',
       '22_50789797_A_G', '22_50789956_A_G', '22_50790558_T_C',
       '22_50790915_G_T', '22_50791201_A_G', '22_50791260_T_C',
       '22_50791513_T_C', '22_50792104_A_C', '22_50792374_G_A',
       '22_50792826_G_A', '22_50793661_C_T', '22_50794255_A_T',
       '22_50794282_A_C', '22_50794538_T_C', '22_50794804_G_C',
       '22_50795071_T_C', '22_50795146_T_A', '22_50795310_C_G',
       '22_50795342_A_G', '22_50795620_G_A', '22_50795806_G_C',
       '22_50795934_C_T', '22_50796278_T_C', '22_50796296_A_G',
       '22_50796448_A_G', '22_50796460_A_G', '22_50797256_A_C',
       '22_50797973_G_A', '22_50798134_C_T', '22_50798212_A_G',
       '22_50798585_A_G', '22_50799169_G_A', '22_50799684_C_A',
       '22_50799845_G_A', '22_50800769_G_C', '22_50800929_A_G',
       '22_50801156_G_C', '22_50801334_A_G', '22_50802074_T_C',
       '22_50802635_T_A', '22_50802642_C_T', '22_50802699_T_G',
       '22_50803110_A_G', '22_50803458_C

In [34]:
get_gene_location(gene_id)

('RPL23AP82', '22', 50798655, 50799123, 1)

In [50]:
test=compute_burdens(data, max_af=0.05, weight_cols=["DISTANCE", "CADD_PHRED", "DNA_LM_influence_score", "MAF_beta_1.25"], window_size=100)


/data/nasif12/home_if12/l_back/sysGen/sc-genetics/src/cellink/tl/_burden_testing.py:59: ImplicitModificationWarning: Trying to modify attribute `.varm` of view, initializing view as actual.
  gdata[['Chromosome', 'Position']] = gdata['Location'].str.split(':', expand=True)
/data/nasif12/home_if12/l_back/sysGen/sc-genetics/src/cellink/tl/_burden_testing.py:60: ImplicitModificationWarning: Trying to modify attribute `.varm` of view, initializing view as actual.
  gdata['Position'] = gdata['Position'].astype(int)
  1%|          | 8/673 [00:25<34:40,  3.13s/it]

Failed to retrieve location for gene ENSG00000273203. Falling back to 'Gene' column.


  3%|▎         | 17/673 [00:52<33:22,  3.05s/it]

Failed to retrieve location for gene ENSG00000272675. Falling back to 'Gene' column.


 11%|█▏        | 76/673 [03:52<30:15,  3.04s/it]

...

 98%|█████████▊| 659/673 [33:39<00:41,  2.96s/it]

Failed to retrieve location for gene ENSG00000226738. Falling back to 'Gene' column.


 99%|█████████▉| 666/673 [34:00<00:20,  2.97s/it]

Failed to retrieve location for gene ENSG00000272940. Falling back to 'Gene' column.



00%|██████████| 673/673 [34:21<00:00,  3.06s/it]

In [61]:
data.gdata.varm["annotations_0"][data.gdata.varm["annotations_0"]["Gene"]=="ENSG00000184319"]

gnomADe_EAS_AF Feature_type  Consequence_intergenic_variant  \
snp_id                                                                         
22_51197576_C_G             NaN   Transcript                               0   
22_51198868_C_T             NaN   Transcript                               0   
22_51198906_A_G             NaN   Transcript                               0   
22_51198998_A_C             NaN   Transcript                               0   
22_51202748_G_A             NaN   Transcript                               0   
22_51208568_T_G             NaN   Transcript                               0   
22_51211031_G_A             NaN   Transcript                               0   
22_51213613_T_C             NaN   Transcript                               0   
22_51216564_C_T             NaN   Transcript                               0   

                PHENO  Consequence_start_lost  gnomADe_AMR_AF  \
snp_id                                                          
22_51197576_C_G     -                       0             NaN   
22_51198868_C_T     -                       0             NaN   
22_51198906_A_G     -                       0             NaN   
22_51198998_A_C     -                       0             NaN   
22_51202748_G_A     -                       0             NaN   
22_51208568_T_G     -                       0             NaN   
22_51211031_G_A     -                       0             NaN   
22_51213613_T_C     -                       0             NaN   
22_51216564_C_T     -                       0             NaN   

                Existing_variation CLIN_SIG  CADD_RAW  gnomADe_NFE_AF  ...  \
snp_id                                                                 ...   
22_51197576_C_G        rs147713773        - -0.140583             NaN  ...   
22_51198868_C_T        rs185017671        -  0.608459             NaN  ...   
22_51198906_A_G          rs6010079        -  0.233827             NaN  ...   
22_51198998_A_C        rs117815989        - -0.565526             NaN  ...   
22_51202748_G_A          rs9616963        -  0.550962             NaN  ...   
22_51208568_T_G        rs148425445        -  0.102244             NaN  ...   
22_51211031_G_A          rs9616968        -  0.153297             NaN  ...   
22_51213613_T_C         rs34726907        - -0.394121             NaN  ...   
22_51216564_C_T          rs9616970        - -0.113869             NaN  ...   

                 Consequence_splice_donor_variant  \
snp_id                                              
22_51197576_C_G                                 0   
22_51198868_C_T                                 0   
22_51198906_A_G                                 0   
22_51198998_A_C                                 0   
22_51202748_G_A                                 0   
22_51208568_T_G                                 0   
22_51211031_G_A                                 0   
22_51213613_T_C                                 0   
22_51216564_C_T                                 0   

                 Consequence_splice_polypyrimidine_tract_variant  \
snp_id                                                             
22_51197576_C_G                                                0   
22_51198868_C_T                                                0   
22_51198906_A_G                                                0   
22_51198998_A_C                                                0   
22_51202748_G_A                                                0   
22_51208568_T_G                                                0   
22_51211031_G_A                                                0   
22_51213613_T_C                                                0   
22_51216564_C_T                                                0   

                 Consequence_splice_region_variant Consequence_stop_gained  \
snp_id                                                                       
22_51197576_C_G                                  0                      

In [62]:
all_res

burden_gene      target_gene             burden_type    pvalue  \
0   ENSG00000100181  ENSG00000100181                DISTANCE  0.763202   
1   ENSG00000100181  ENSG00000100181              CADD_PHRED  0.790973   
2   ENSG00000100181  ENSG00000100181  DNA_LM_influence_score  0.552800   
3   ENSG00000100181  ENSG00000100181           MAF_beta_1.25  0.854610   
0   ENSG00000237438  ENSG00000237438                DISTANCE  0.568767   
..              ...              ...                     ...       ...   
3   ENSG00000100288  ENSG00000100288           MAF_beta_1.25  0.083261   
0   ENSG00000079974  ENSG00000079974                DISTANCE  1.000000   
1   ENSG00000079974  ENSG00000079974              CADD_PHRED  1.000000   
2   ENSG00000079974  ENSG00000079974  DNA_LM_influence_score  1.000000   
3   ENSG00000079974  ENSG00000079974           MAF_beta_1.25  1.000000   

        beta     cell_type chrom  pvalue_corrected  significant  
0   0.000127        CD8 ET    22      19290.695395        False  
1  -0.000112        CD8 ET    22      19992.627004        False  
2  -0.000252        CD8 ET    22      13972.564388        False  
3  -0.000078        CD8 ET    22      21601.110499        False  
0   0.000099        CD8 ET    22      14376.159711        False  
..       ...           ...   ...               ...          ...  
3  -0.031435  Erythrocytes    22       2104.513138        False  
0        NaN  Erythrocytes    22      25276.000000        False  
1        NaN  Erythrocytes    22      25276.000000        False  
2        NaN  Erythrocytes    22      25276.000000        False  
3        NaN  Erythrocytes    22      25276.000000        False  

[25276 rows x 9 columns]